In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input

In [2]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1000, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1000,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)
        y_batch = np.eye(2)[self.y[batch_indices]]
        return X_batch, y_batch
    
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")
        
class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        """
        Early stopping within the same epoch based on a monitored metric.
        Args:
            monitor: Metric to monitor ('loss', 'accuracy', etc.).
            threshold: Threshold value for stopping (e.g., loss < 0.1).
            patience: Number of batches to wait for improvement before stopping.
        """
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0 

batch_early_stopping_callback = BatchEarlyStopping(
    monitor='loss',     
    threshold=0.1,       
    patience=2          
)

def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  

    sequence_input = Input(shape=(seq_length, height, width, channels))
    
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    lstm_out = LSTM(128)(flattened_features)
    lstm_out = Dropout(0.5)(lstm_out)

    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.5)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

indices = np.arange(1000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', train_indices, batch_size=10)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', val_indices, batch_size=10)

checkpoint_dir = './Final_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',  
    save_best_only=False,
    save_weights_only=False,  
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2: 
        return lr
    return lr * 0.9 

lr_callback = LearningRateScheduler(scheduler)

step_timer_callback = StepTimerCallback()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1  
)

In [3]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [4]:
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(1000, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(1000,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  
        y_batch = np.eye(2)[self.y[batch_indices]]  
        return X_batch, y_batch

class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")

class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None and current_value < self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                self.model.stop_training = True
        else:
            self.wait = 0

batch_early_stopping_callback = BatchEarlyStopping(monitor='loss', threshold=0.1, patience=2)

indices = np.arange(1000)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', train_indices, batch_size=10)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_final_15k.dat', r'E:\PosePerfect\Dataset Creation\y_final_15k.dat', val_indices, batch_size=10)

checkpoint_dir = './Final_Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    return lr * 0.9

lr_callback = LearningRateScheduler(scheduler)

latest_checkpoint = r'E:\PosePerfect\Model\Final_Checkpoints\model_epoch_09_val_loss_0.65.keras'

cnn_lstm_model = load_model(latest_checkpoint)

cnn_lstm_model.summary()

cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10, 
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, StepTimerCallback(), batch_early_stopping_callback],
    verbose=1
)


In [5]:
val_loss, val_accuracy = cnn_lstm_model.evaluate(val_gen, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = np.concatenate([val_gen[i][1] for i in range(len(val_gen))])
y_true = np.argmax(y_true, axis=1)  

y_pred = cnn_lstm_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1']))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [7]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

model_path = r'E:\PosePerfect\Model\Final_Checkpoints\model_epoch_10_val_loss_0.44.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_7.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")


In [8]:
example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

In [10]:
import os

new_save_folder = r'E:\PosePerfect\Model\Final_Models'
os.makedirs(new_save_folder, exist_ok=True)

new_model_path = os.path.join(new_save_folder, 'modelv2.keras')
cnn_lstm_model.save(new_model_path)

print(f"Model saved successfully to {new_model_path}!")

In [11]:
model_path = r'E:\PosePerfect\Model\MoreCheckpoints\model_epoch_10_val_loss_0.60.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

new_save_folder = r'E:\PosePerfect\Model\Final_Models'

new_model_path = os.path.join(new_save_folder, 'modelv1.keras')
cnn_lstm_model.save(new_model_path)

print(f"Model saved successfully to {new_model_path}!")


In [12]:
model_path = r'E:\PosePerfect\Model\Final_Models\modelv2.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

In [13]:
model_path = r'E:\PosePerfect\Model\Final_Models\modelv1.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_1.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")

In [11]:
model_path = r'E:\PosePerfect\Model\Final_Checkpoints\model_epoch_10_val_loss_0.44.keras'
cnn_lstm_model = load_model(model_path)
print("Model loaded successfully!")

example_input = np.memmap(
    r'E:\PosePerfect\Model\Process Example\X_exam_16.dat',
    dtype='float32',
    mode='r',
    shape=(2, 15, 224, 224, 3)
)

example_input_writable = np.array(example_input) 

example_input_preprocessed = preprocess_input(example_input_writable)
print("Input preprocessed successfully!")

print(f"Input Shape: {example_input_preprocessed.shape}")
print(f"Input Summary: Min={example_input_preprocessed.min()}, Max={example_input_preprocessed.max()}, Mean={example_input_preprocessed.mean()}")

predictions = cnn_lstm_model.predict(example_input_preprocessed)
predicted_class = np.argmax(predictions, axis=1)

print("Raw Predictions:", predictions)
print("Predicted Class Probabilities:", predictions[0]) 
print(f"Predicted Class: {predicted_class}")